In [47]:
import os,shutil
from keras.preprocessing.image import ImageDataGenerator
import csv
import numpy as np
from matplotlib import pyplot as plt
from keras import layers
from keras import models

csv_file=csv.reader(open('train.csv','r'))
label465=np.zeros(shape=(465))
label400=np.zeros(shape=(400))
label65=np.zeros(shape=(65))

n=0
for m in csv_file:
    label465[n]=m[1]
    n=n+1
for o in range(0,400):
    label400[o]=label465[o]
for p in range(0,65):
    label65[p]=label465[p+400]
labelswap=np.append(label400,label400)
csv_file=csv.reader(open('test.csv','r'))
label117=np.zeros(shape=(117))
q=0
for p in csv_file:
    label117[q]=p[1]
    q=q+1
for i in range(0,117):
    if(label117[i]>0.5):
        label117[i]=1
    else:
        label117[i]=0
label4=np.append(label400,label117)

In [48]:
import random
path='train_val'
image_files=sorted(os.listdir(path))
train465=np.zeros(shape=(465,30,30,30))
train400=np.zeros(shape=(400,30,30,30))
train65=np.zeros(shape=(65,30,30,30))

samplepair=np.zeros(shape=(400,30,30,30))
labelsample=np.zeros(shape=(400))


mixup=np.zeros(shape=(400,30,30,30))
labelmixup=np.zeros(shape=(400))

swap=np.swapaxes(train400,1,2)
j=0
for i in image_files:
    tmp = np.load(os.path.join(path,i))
    train465[j]=tmp['voxel'][35:65,35:65,35:65]/255.0
    j=j+1
for q in range(0,400):
    train400[q]=train465[q]
    
for r in range(0,65):
    train65[r]=train465[r+400]

for s in range(0,400):
    a=random.randint(0,399)
    b=random.randint(0,399)
    samplepair[s]=0.5*train400[a]+0.5*train400[b]
    labelsample[s]=label400[a]
for s in range(0,400):
    e=random.randint(0,399)
    f=random.randint(0,399)
    mixup[s]=0.5*train400[e]+0.5*train400[f]
    labelmixup[s]=0.5*label400[e]+0.5*label400[f]


train4=np.append(train400,c)
train4=train4.reshape(517,30,30,30,1)
samplefinal=np.append(samplepair,train400)
labelfinal=np.append(labelsample,label400)
samplefinal=samplefinal.reshape(800,30,30,30,1)

mixupfinal=np.append(mixup,train400)
mixupfinal=mixupfinal.reshape(800,30,30,30,1)
swapfinal=np.append(swap,train400)
swapfinal=swapfinal.reshape(800,30,30,30,1)
train465=train465.reshape(465,30,30,30,1)
train400=train400.reshape(400,30,30,30,1)
train65=train65.reshape(65,30,30,30,1)   



In [49]:
from keras import regularizers
model1=models.Sequential()
model1.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(30,30,30,1)))

model1.add(layers.MaxPooling3D((2,2,2)))
model1.add(layers.Conv3D(64,(3,3,3),activation='relu'))

model1.add(layers.MaxPooling3D((2,2,2)))


model1.add(layers.Flatten())
model1.add(layers.Dropout(0.4))
model1.add(layers.Dense(1024,activation='relu'))
model1.add(layers.Dense(1,activation='sigmoid'))

from keras import optimizers

model1.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=0.0001),metrics=['accuracy'])

In [50]:
from keras.callbacks import EarlyStopping
early_stopping2=EarlyStopping(monitor='val_loss',patience=4,mode='min')
model1.fit(train4,label4,epochs=1,batch_size=32,validation_split=0.2,callbacks=[early_stopping2])#epochs手动调试

Train on 413 samples, validate on 104 samples
Epoch 1/1
413/413 [==============================] - 11s 28ms/step - loss: 0.7007 - acc: 0.5448 - val_loss: 0.6077 - val_acc: 0.7115


In [51]:
eva=model1.evaluate(train65,label65)
print(eva)

65/65 [==============================] - 0s 7ms/step
[0.6693436164122362, 0.5846153846153846]


In [52]:
model1.fit(samplefinal,labelfinal,epochs=5,batch_size=32,validation_split=0.2,callbacks=[early_stopping2])

Train on 640 samples, validate on 160 samples
Epoch 1/5
640/640 [==============================] - 15s 24ms/step - loss: 0.6741 - acc: 0.5844 - val_loss: 0.6418 - val_acc: 0.5938
Epoch 2/5
640/640 [==============================] - 15s 24ms/step - loss: 0.6554 - acc: 0.6375 - val_loss: 0.6576 - val_acc: 0.5563
Epoch 3/5
640/640 [==============================] - 15s 23ms/step - loss: 0.6448 - acc: 0.6297 - val_loss: 0.6406 - val_acc: 0.5687
Epoch 4/5
640/640 [==============================] - 16s 26ms/step - loss: 0.6182 - acc: 0.6672 - val_loss: 0.6408 - val_acc: 0.5813
Epoch 5/5
640/640 [==============================] - 15s 23ms/step - loss: 0.6082 - acc: 0.6953 - val_loss: 0.6154 - val_acc: 0.6562


In [46]:
model1.save('my_model4.h5')
modelsave==models.load_model('my_model4.h5')

NameError: name 'modelsave' is not defined

In [15]:
eva=model1.evaluate(train65,label65)
print(eva)

65/65 [==============================] - 0s 7ms/step
[0.7057061708890475, 0.5538461538461539]


In [21]:
c=np.zeros(shape=(117,30,30,30))
paths='test'
test_files=sorted(os.listdir(paths))
p=0
for q in test_files:
    
    tmp = np.load(os.path.join(paths,q))
    c[p]=tmp['voxel'][35:65,35:65,35:65]/255.0
    p=p+1

c=c.reshape((117,30,30,30,1))


score=model1.predict(c)
print(score)

[[0.7168231 ]
 [0.3976467 ]
 [0.47626   ]
 [0.9341206 ]
 [0.42453057]
 [0.67844194]
 [0.702422  ]
 [0.59924906]
 [0.45456633]
 [0.5081272 ]
 [0.47062817]
 [0.83767134]
 [0.6994971 ]
 [0.41104567]
 [0.33384046]
 [0.56978536]
 [0.7479873 ]
 [0.40999103]
 [0.67682844]
 [0.65185046]
 [0.81583565]
 [0.66584885]
 [0.5642184 ]
 [0.49157634]
 [0.8724612 ]
 [0.9153406 ]
 [0.67426133]
 [0.5282696 ]
 [0.4873438 ]
 [0.46090215]
 [0.7588322 ]
 [0.50868523]
 [0.63282233]
 [0.54059243]
 [0.4413819 ]
 [0.55233514]
 [0.76761603]
 [0.65770173]
 [0.82338536]
 [0.4182033 ]
 [0.7609011 ]
 [0.76655805]
 [0.489962  ]
 [0.7277554 ]
 [0.79878   ]
 [0.8230746 ]
 [0.4411409 ]
 [0.43290237]
 [0.5559587 ]
 [0.59411407]
 [0.50527453]
 [0.50793684]
 [0.6354553 ]
 [0.609521  ]
 [0.53169334]
 [0.58743185]
 [0.83007675]
 [0.8209094 ]
 [0.66582364]
 [0.4667226 ]
 [0.37256026]
 [0.7285701 ]
 [0.40157145]
 [0.8115135 ]
 [0.59788585]
 [0.5161753 ]
 [0.6173512 ]
 [0.49215698]
 [0.4914931 ]
 [0.81761694]
 [0.5006844 ]
 [0.56

In [45]:
model1.save('my_model1.h5')
model1save=keras.models.load_model('my_model1.h5')

In [33]:
model2=models.Sequential()
model2.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(30,30,30,1)))

model2.add(layers.MaxPooling3D((2,2,2)))
model2.add(layers.Conv3D(64,(3,3,3),activation='relu'))

model2.add(layers.MaxPooling3D((2,2,2)))


model2.add(layers.Flatten())
model2.add(layers.Dropout(0.4))
model2.add(layers.Dense(1024,activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))

from keras import optimizers

model2.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=0.0001),metrics=['accuracy'])

In [34]:
model2.fit(train400,label400,epochs=3,batch_size=32,validation_split=0.2)

Train on 320 samples, validate on 80 samples
Epoch 1/3
320/320 [==============================] - 8s 25ms/step - loss: 0.7077 - acc: 0.4969 - val_loss: 0.6869 - val_acc: 0.5250
Epoch 2/3
320/320 [==============================] - 6s 20ms/step - loss: 0.6666 - acc: 0.6312 - val_loss: 0.7049 - val_acc: 0.5125
Epoch 3/3
320/320 [==============================] - 6s 20ms/step - loss: 0.6620 - acc: 0.6031 - val_loss: 0.6769 - val_acc: 0.5250


In [137]:
model2.fit(mixupfinal,labelmixupfinal,epochs=1,batch_size=32,validation_split=0.2,callbacks=[early_stopping2])

Train on 640 samples, validate on 160 samples
Epoch 1/1
640/640 [==============================] - 14s 22ms/step - loss: 0.4710 - acc: 0.5563 - val_loss: 0.5364 - val_acc: 0.7312


In [138]:
eva2=model2.evaluate(train65,label65)
print(eva2)

65/65 [==============================] - 0s 6ms/step
[0.7901287106367257, 0.5384615384615384]


In [65]:
model2.save('my_model2.h5')
model2save=keras.models.load_model('my_model2.h5')

In [92]:
score2=model2save.predict(c)
print(score2)

[[0.5409618 ]
 [0.48130608]
 [0.5023697 ]
 [0.918199  ]
 [0.38078427]
 [0.68815327]
 [0.6221941 ]
 [0.61075795]
 [0.5889826 ]
 [0.6157326 ]
 [0.61123925]
 [0.8689493 ]
 [0.6871264 ]
 [0.3722501 ]
 [0.53952265]
 [0.56172264]
 [0.7550874 ]
 [0.4861538 ]
 [0.5511708 ]
 [0.6257304 ]
 [0.8069252 ]
 [0.6623259 ]
 [0.59943384]
 [0.6189102 ]
 [0.8369938 ]
 [0.8641398 ]
 [0.61751497]
 [0.5359121 ]
 [0.42753834]
 [0.49731904]
 [0.72360295]
 [0.5285295 ]
 [0.6672171 ]
 [0.678749  ]
 [0.44511956]
 [0.5225738 ]
 [0.7938489 ]
 [0.67486864]
 [0.78453636]
 [0.41087282]
 [0.74758065]
 [0.75141054]
 [0.61515045]
 [0.79998815]
 [0.80905217]
 [0.79256773]
 [0.5720327 ]
 [0.38695008]
 [0.5462779 ]
 [0.5436133 ]
 [0.5594635 ]
 [0.54089814]
 [0.6146288 ]
 [0.67220026]
 [0.5271485 ]
 [0.53931737]
 [0.80304366]
 [0.8388326 ]
 [0.6118367 ]
 [0.61292017]
 [0.46928132]
 [0.74340546]
 [0.48434404]
 [0.7945712 ]
 [0.6675482 ]
 [0.50483644]
 [0.5778162 ]
 [0.55774546]
 [0.6322199 ]
 [0.7565576 ]
 [0.5195486 ]
 [0.53

In [98]:
a=0
for i in range(0,117):
    if (((score2[i]<0.3)&(score[i]<0.53))|((score2[i]>0.7)&(score[i]>0.7))):
        a=a+1
print(a)
b=0
for i in range(0,117):
    if (((score2[i]<0.3)&(score[i]>0.7))|((score2[i]>0.7)&(score[i]<0.3))):
        b=b+1
print(b)

26
0


In [23]:
model3=models.Sequential()
model3.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(30,30,30,1)))

model3.add(layers.MaxPooling3D((2,2,2)))
model3.add(layers.Conv3D(64,(3,3,3),activation='relu'))

model3.add(layers.MaxPooling3D((2,2,2)))


model3.add(layers.Flatten())
model3.add(layers.Dropout(0.4))
model3.add(layers.Dense(1024,activation='relu'))
model3.add(layers.Dense(1,activation='sigmoid'))

from keras import optimizers

model3.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=0.0001),metrics=['accuracy'])

In [24]:
model3.fit(train400,label400,epochs=5,batch_size=32,validation_split=0.2)

Train on 320 samples, validate on 80 samples
Epoch 1/5
320/320 [==============================] - 8s 25ms/step - loss: 0.6958 - acc: 0.5375 - val_loss: 0.7269 - val_acc: 0.5000
Epoch 2/5
320/320 [==============================] - 6s 20ms/step - loss: 0.6785 - acc: 0.5719 - val_loss: 0.6755 - val_acc: 0.5000
Epoch 3/5
320/320 [==============================] - 6s 20ms/step - loss: 0.6386 - acc: 0.6594 - val_loss: 0.7303 - val_acc: 0.5125
Epoch 4/5
320/320 [==============================] - 6s 20ms/step - loss: 0.6306 - acc: 0.6437 - val_loss: 0.6862 - val_acc: 0.5000
Epoch 5/5
320/320 [==============================] - 6s 20ms/step - loss: 0.6133 - acc: 0.6437 - val_loss: 0.6856 - val_acc: 0.4625


In [26]:
model3.fit(swapfinal,labelswap,epochs=5,batch_size=32,validation_split=0.2)

Train on 640 samples, validate on 160 samples
Epoch 1/5
640/640 [==============================] - 13s 20ms/step - loss: 0.6396 - acc: 0.6469 - val_loss: 0.6286 - val_acc: 0.6250
Epoch 2/5
640/640 [==============================] - 13s 20ms/step - loss: 0.6238 - acc: 0.6578 - val_loss: 0.6249 - val_acc: 0.6687
Epoch 3/5
640/640 [==============================] - 13s 20ms/step - loss: 0.5949 - acc: 0.7125 - val_loss: 0.6176 - val_acc: 0.6687
Epoch 4/5
640/640 [==============================] - 13s 20ms/step - loss: 0.5688 - acc: 0.7312 - val_loss: 0.6747 - val_acc: 0.5687
Epoch 5/5
640/640 [==============================] - 13s 20ms/step - loss: 0.5592 - acc: 0.7203 - val_loss: 0.6220 - val_acc: 0.6500


In [27]:
eva2=model3.evaluate(train65,label65)
print(eva2)

65/65 [==============================] - 0s 6ms/step
[0.6956161123055679, 0.6]


In [28]:
score3=model3.predict(c)
print(score3)

[[0.4188062 ]
 [0.34298444]
 [0.5488532 ]
 [0.8065519 ]
 [0.5803019 ]
 [0.5090016 ]
 [0.72942114]
 [0.4160618 ]
 [0.36754084]
 [0.360031  ]
 [0.4402225 ]
 [0.57347035]
 [0.5905064 ]
 [0.36552456]
 [0.53267425]
 [0.46041775]
 [0.611719  ]
 [0.28863388]
 [0.45345825]
 [0.5227828 ]
 [0.65864825]
 [0.54706   ]
 [0.65138745]
 [0.43694803]
 [0.7292552 ]
 [0.7238175 ]
 [0.76707387]
 [0.38684106]
 [0.254462  ]
 [0.5109318 ]
 [0.6364933 ]
 [0.39824954]
 [0.5259981 ]
 [0.3350095 ]
 [0.37609282]
 [0.4298329 ]
 [0.60221326]
 [0.5267599 ]
 [0.52877796]
 [0.2840783 ]
 [0.5645223 ]
 [0.57081604]
 [0.4337801 ]
 [0.40690148]
 [0.5628629 ]
 [0.6489444 ]
 [0.42828202]
 [0.34055758]
 [0.40908062]
 [0.3657067 ]
 [0.4989976 ]
 [0.3658449 ]
 [0.44947672]
 [0.44151157]
 [0.36311543]
 [0.5888907 ]
 [0.7533396 ]
 [0.6665604 ]
 [0.4709901 ]
 [0.37159598]
 [0.43439937]
 [0.4517023 ]
 [0.28601   ]
 [0.6424357 ]
 [0.44268164]
 [0.4632809 ]
 [0.36304837]
 [0.5121583 ]
 [0.4373169 ]
 [0.75526   ]
 [0.4545716 ]
 [0.43

In [42]:
model3.save('my_model3.h5')
model3save=models.load_model('my_model3.h5')
model2save=models.load_model('my_model2.h5')
model1save=models.load_model('my_model1.h5')

In [43]:
eva3=model3save.evaluate(train65,label65)
print(eva3)
eva2=model2save.evaluate(train65,label65)
print(eva2)
eva1=model1save.evaluate(train65,label65)
print(eva1)


65/65 [==============================] - 1s 11ms/step
[0.6956161123055679, 0.6]
65/65 [==============================] - 1s 10ms/step
[0.6956469609187199, 0.6153846153846154]
65/65 [==============================] - 1s 10ms/step
[0.7323742472208463, 0.6153846153846154]


In [1]:
score1=model1save.predict(c)
score2=model2save.predict(c)
score3=model3save.predict(c)
scorefinal=np.zeros(shape=(117))
n=0
for i in range(0,117):
    a=score1[i]
    b=score2[i]
    d=score3[i]
    if(max(a,b,d)>0.55):
        scorefinal[i]=max(a,b,d)
        n=n+1
    if(min(a,b,d)<0.45):
        scorefinal[i]=min(a,b,d)
        n=n+1
print(n)
for i in range(0,117):
    print(scorefinal[i])
    

NameError: name 'model1save' is not defined

In [24]:
from keras import regularizers
model4=models.Sequential()
model4.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(30,30,30,1)))

model4.add(layers.MaxPooling3D((2,2,2)))
model4.add(layers.Conv3D(64,(3,3,3),activation='relu'))

model4.add(layers.MaxPooling3D((2,2,2)))


model4.add(layers.Flatten())
model4.add(layers.Dropout(0.5))
model4.add(layers.Dense(1024,activation='relu'))
model4.add(layers.Dense(1,activation='sigmoid'))

from keras import optimizers

model4.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=0.0001),metrics=['accuracy'])

In [27]:
from keras.callbacks import EarlyStopping
early_stopping2=EarlyStopping(monitor='val_loss',patience=4,mode='min')
model4.fit(train4,label4,epochs=5,batch_size=32,validation_split=0.2,callbacks=[early_stopping2])

Train on 413 samples, validate on 104 samples
Epoch 1/5
413/413 [==============================] - 10s 24ms/step - loss: 0.6113 - acc: 0.6586 - val_loss: 0.5364 - val_acc: 0.8750
Epoch 2/5
413/413 [==============================] - 10s 24ms/step - loss: 0.5951 - acc: 0.7046 - val_loss: 0.5423 - val_acc: 0.8077
Epoch 3/5
413/413 [==============================] - 10s 24ms/step - loss: 0.5667 - acc: 0.7361 - val_loss: 0.5691 - val_acc: 0.7212
Epoch 4/5
413/413 [==============================] - 10s 24ms/step - loss: 0.5537 - acc: 0.7094 - val_loss: 0.5088 - val_acc: 0.7019
Epoch 5/5
413/413 [==============================] - 10s 24ms/step - loss: 0.5316 - acc: 0.7409 - val_loss: 0.5068 - val_acc: 0.8269


In [28]:
eva2=model4.evaluate(train65,label65)
print(eva2)

65/65 [==============================] - 0s 7ms/step
[0.7291619245822613, 0.49230769230769234]
